# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [1]:
def fastModularExponentation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m

### Skąd wziąć dużą liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**

Test probabilistyczny, np. Rabina-Millera.

In [2]:
import random

def rm_test(d, n):
    a = random.randint(2, n - 2)
    x = fastModularExponentation(a, d, n)

    if x == 1 or x == n - 1:
        return True

    while d != n - 1:
        x = fastModularExponentation(x, 2, n)
        d *= 2

        if x == 1:
            return False
        if x == n - 1:
            return True

    return False

In [3]:
def isPrime3(n, k):
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False    # speedup

    s = 0
    d = n-1
    while d % 2 == 0:
        s += 1
        d //= 2

    #tutaj napisz swój kod
    for i in range(k):
        if rm_test(d, n) == False:
            return False

    return True

for i in range(65500,65600):            
    if(isPrime3(i,10)): print(i)

65519
65521
65537
65539
65543
65551
65557
65563
65579
65581
65587
65599


### Obliczanie elementu odwrotnego w ciele skończonym
1. Algorytm Euklidesa - znajdowanie największego wspólnego dzielnika
2. Rozszerzony Algorytm Euklidesa - obliczanie elementu odwrotnego względem mnożenia w ciele skończonym

Przeanalizuj poniższy kod.

In [4]:

## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

# wersja rekurencyjna
def gcdExtendedRecursion(a, b):  
    # przypadek trywialny
    if a == 0 :   
        return b,1,0             
    gcd,x1,y1 = gcdExtendedRecursion(b%a, a) #rekurencyjny Euklides      
   
    # sumowanie powrotne  
    y = x1 - (b//a) * y1  
    x = y1  
    
    return gcd,x,y 

#wersja iteracyjna
def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m


a, b = 154,801
g, x, y = gcdExtendedRecursion(a, b)  
print("gcd(", a , "," , b, ") = ", g, x, y)
print("Element odwrotny do ", a, "modulo", b, ":", y%b)

print(findModInverse(154,801))

gcd( 154 , 801 ) =  1 5 -26
Element odwrotny do  154 modulo 801 : 775
775


## Generowanie kluczy w kryptosystemie RSA
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu.
### Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [5]:
first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
                     31, 37, 41, 43, 47, 53, 59, 61, 67,
                     71, 73, 79, 83, 89, 97, 101, 103,
                     107, 109, 113, 127, 131, 137, 139,
                     149, 151, 157, 163, 167, 173, 179,
                     181, 191, 193, 197, 199, 211, 223,
                     227, 229, 233, 239, 241, 251, 257,
                     263, 269, 271, 277, 281, 283, 293,
                     307, 311, 313, 317, 331, 337, 347, 349]

def get_low_level_prime(n):
    while True:
        candidate = random.randint(2**(n - 1), 2**n)

        for div in first_primes_list:
            if candidate % div == 0 and div**2  <= candidate:
                break
        else:
            return candidate

#napisz funkcję generującą liczbę pierwszą
def generatePrime(keysize):
    #napisz swój kod tutaj
    while True:
        candidate = random.randint(2**(keysize - 1), 2**keysize)
        if isPrime3(candidate, 200):
            return candidate

print(generatePrime(2048))

23238268445690454681162962244876934481249010407784893701625365470180913141336765955441433691877666013777728130385117310965295416920096979667641546569024758408208458167607195607855040360451617541217627384323090376116105613457288200849019716917982124831158322222563598589934498769650239062657291183863413044614371316766139950576581829773379800486380547264879154654076186935485225492189456698127116448052534588295185145634271823132343930477875217005165325998909773212703808332548682531988800694626216243097432065313094081125484296749485897385991462742691006645057336985315303283251594924403070413310622454645352442094407


## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [6]:
import random, sys, os

def generateKey(keySize):
    p = generatePrime(keySize)
    q = generatePrime(keySize)

    n = p * q

    e = (p - 1) * (q - 1)
    while gcd(e, (p - 1) * (q - 1)) != 1:
        e = random.randint(2 ** (keySize - 1), 2 ** keySize -1)

    d = findModInverse(e, (p - 1) * (q - 1))

    publicKey = (n, e)
    privateKey = (n, d)

    return (publicKey, privateKey)

pub,priv = generateKey(16)
print(pub,priv)

(1870759207, 43567) (1870759207, 1053651343)


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [7]:
def encrypt(message, modulus, exp):
    message_encrypted = pow(ord(message), exp, modulus)
  
    return message_encrypted

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [8]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = chr(pow(message_encrypted, exp, modulus))

    return ('').join(message_ascii)

In [9]:
# generujemy klucz
pub_key, priv_key = generateKey(2048)

# wiadomośc do zaszyfrowania
message = 'm'
print("Wiadomość: " + message)

# szyfrujemy
n, e = pub_key
encrypted_message = encrypt(message, n, e)
print('Wiadomośc zaszyfrowana: ' + str(encrypted_message))

_, d = priv_key
# rozszyfrowujemy
print('Wiadomośc odszyfrowana: ' + decrypt(encrypted_message, n, d))

Wiadomość: m
Wiadomośc zaszyfrowana: 293761793877950546578383400075606331690087209394645450433278918460271716438280205968877414501726443573274996998877361629462764492282773784109843732571068663644818801154644718656346497048550643539233692415749061235040454544211448166644006166300836592666552150549516007938539780802580105027111408916092019223974931966782634522233913219818461833954731854272092494496726416367727578845035931791159410945398076427804605190304331934686190371698783313582629270335391213007402183977350870997124092130237374852578349209096382286101484587831341980978043782486801331026408990661371124750742773222879935277354456376098977621376658689152744321125923596435377726259155813407966701253503966562179036803507141157489039985998584285598955238581758800951010480826072272880855941545398901682371686365199547228225319761605183375093461927201567089616721878302387000267408399460523460399917018368968008920990365912918904741165997929383053925704960832971651040078169880873326778037667941

## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


## Zadanie (dla chętnych)
1. Zademonstruj deformowalność podręcznikowego szyfrowania RSA
2. Zaimplementuj dopełnienie OAEP (_Optimal Assymmetric Encryption Padding_)
